In [255]:
# ALPLA demo- Implementation with a small scale dataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load measurement file:
measurement = np.load('dataset/meas_symm_2.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']

# Initialize the scaler
scaler = MinMaxScaler()
# scaler = StandardScaler()

# Initialize the One-Class SVM
# ocsvm = OneClassSVM(kernel='linear', gamma='scale', nu=0.1) # tp=1183 fp=1321   bf=100 mw=500
# ocsvm = OneClassSVM(kernel='linear', gamma='scale', nu=0.2) # tp=850  fp=1019   bf=100 mw=500
# ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.5)  # tp=338  fp=464    bf=100 mw=500
# ocsvm = OneClassSVM(kernel='linear', gamma='scale', nu=0.1)   # tp=1246 fp=1376   bf=200 mw=200

# ocsvm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.1)   # tp=613 fp=326   bf=200 mw=200
# ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)   # tp=949 fp=933 bf=200 mw=200


ocsvm = OneClassSVM(kernel='linear', gamma='scale', nu=0.01) # tp=1221  fp=1360   bf=200 mw=200

# ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.5)  # tp=338  fp=464    bf=100 mw=100

# ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.1)  # tp=1183 fp=1321   bf=100 mw=500

# ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.005)
# ocsvm = OneClassSVM(kernel='linear', gamma='auto', nu=0.005)
# ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)  # Switching to RBF kernel

In [256]:
def servesImportant64Samples(real, imag):

    # Number of signals
    num_signals = real.shape[0]  # 3 in this case
    
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    img_phase_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        phase = np.angle(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        phase_part_focus = phase[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        img_phase_parts.append(phase_part_focus)

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)
    img_phase_parts = np.array(img_phase_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts, img_phase_parts

In [257]:
def update_features(features, new_cir, max_window_size=100, remove_count = 10):
    # print(features.shape[0])
    # If the number of rows (samples) exceeds the max window size, remove the oldest sample
    if features.shape[0] >= max_window_size:
        # Remove the oldest CIR (first row)
        features = np.delete(features, np.s_[:remove_count], axis=0)
    
    # Append the new CIR to the end
    updated_features = np.vstack([features , new_cir])
    # print(updated_features.shape)
    return updated_features

In [258]:
# def scale_features(real_part, imag_part, mag_part, scaler=None):
#     """
#     This function scales the real, imaginary, and magnitude parts of the CIR features together.
#     """
#     # Combine the real, imaginary, and magnitude parts into a single feature matrix
#     combined_features = np.column_stack((real_part, imag_part, mag_part))
    
#     # If scaler is provided, use it; otherwise, fit a new scaler
#     if scaler is None:
#         scaler = MinMaxScaler()
#         scaler.fit(combined_features)
    
#     # Transform the features using the scaler
#     scaled_features = scaler.transform(combined_features)
    
#     return scaled_features, scaler

In [259]:
#------------------ Spliting the data into training and test sets ------------------
dataset_slice = data['cirs']
# print(dataset_slice[])
# Split data into training and test sets
X_train, X_test = train_test_split(dataset_slice, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(7036, 15, 251, 2)
(1759, 15, 251, 2)


In [260]:
# init_real_251 = X_train[:7000, 3, :, 0]
# init_imag_251 = X_train[:7000, 3, :, 1]
# # init_mag = np.abs(init_real + 1j * init_imag)

# init_real, init_imag, init_mag = servesImportant64Samples(init_real_251, init_imag_251)

# init_real_scaled = scaler.fit_transform(init_real).flatten()
# init_imag_scaled = scaler.fit_transform(init_imag).flatten()
# init_mag_scaled = scaler.fit_transform(init_mag).flatten()

# init_features = np.column_stack((init_real_scaled, init_imag_scaled, init_mag_scaled)).reshape(7000, 192)

# ocsvm.fit(init_features)

In [261]:
alice_real_251 = X_train[:50, 3, :, 0]
alice_imag_251 = X_train[:50, 3, :, 1]

# took important 64 samples
alice_real, alice_imag, alice_mag, alice_phase = servesImportant64Samples(alice_real_251, alice_imag_251)

# feature set
combined_train_features = np.column_stack((alice_real, alice_imag, alice_mag, alice_phase))

# fit the scaler to data
scaler.fit(combined_train_features)
# transform the features using the scaler
scaled_train_features = scaler.transform(combined_train_features)

# train the ocsvm model
ocsvm.fit(scaled_train_features)


alice_features = scaled_train_features

In [262]:


true_labels = []
predictions = []
# Buffer to hold positive samples before retraining
positive_sample_buffer = []
batch_size = 500  # Retrain after collecting 10 new positive samples

selected_channel = [3,6]

# # count = 0
# for cir in range(1):
#     for channel in selected_channel:


# for cir in range(150):
for cir in range(X_test.shape[0]):
    for channel in selected_channel:
        
        # Determine the true label based on the channel
        if channel == 3:
            true_label = 1  # Legitimate user (Alice)
        else:
            true_label = -1  # Illegitimate user (Not Alice)
        
        true_labels.append(true_label)
        
        # Extract the current test CIR
        incoming_real_251 = X_test[cir, channel, :, 0].reshape(1, -1)
        incoming_imag_251 = X_test[cir, channel, :, 1].reshape(1, -1)

        incoming_real, incoming_imag, incoming_mag, incoming_phase = servesImportant64Samples(incoming_real_251, incoming_imag_251)

        combine_test_features = np.column_stack((incoming_real, incoming_imag, incoming_mag, incoming_phase))
        
        combine_test_features_scaled = scaler.transform(combine_test_features)
        
        # Predict using the OCC-SVM
        prediction = ocsvm.predict(combine_test_features_scaled)
        predictions.append(prediction[0])
        
        # Use decision_function and apply a custom threshold
        # decision_scores = ocsvm.decision_function(combine_test_features_scaled)  # Get decision scores
        # threshold = -0.2  # Custom threshold, adjust this value to tune FP/TP trade-off
        # prediction = (decision_scores >= threshold).astype(int)  # Apply custom threshold
        # predictions.append(prediction[0])
        
        if prediction == 1:
            # Add new positive sample to buffer
            positive_sample_buffer.append(combine_test_features_scaled)
            
            # print(np.vstack(positive_sample_buffer).shape)
            # Retrain the model in batches
            if len(positive_sample_buffer) >= batch_size:
                # np.vstack() stacks arrays in sequence vertically (row wise)
                # alice_features = update_features(alice_features, combine_test_features_scaled, max_window_size=100)
                # print(np.vstack(positive_sample_buffer).shape)
                print(alice_features.shape)
                alice_features = update_features(alice_features, np.vstack(positive_sample_buffer), 500, 200)
                print(alice_features.shape)
                # print()
                
                # print(alice_features.shape)
                ocsvm.fit(alice_features)
                positive_sample_buffer.clear()  # Reset buffer
            

(50, 256)
(550, 256)
(550, 256)
(850, 256)
(850, 256)
(1150, 256)
(1150, 256)
(1450, 256)
(1450, 256)
(1750, 256)


In [263]:
# Calculate confusion matrix
print(f"\nTesting set size: {X_test.shape[0]}")

tn, fp, fn, tp = confusion_matrix(true_labels, predictions, labels=[-1, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")
    


Testing set size: 1759
tp: 1398
tn: 302
fp: 1457
fn: 361
MDR: 0.8283115406480955
FAR: 0.20523024445707788
AR: 0.4832291074474133
